# Web2py - El núcleo
## Opciones de la línea de comandos
```cmd
$ python web2py.py -h
Usage: python web2py.py

web2py Web Framework startup script. ATTENTION: unless a password
is specified (-a 'passwd'), web2py will attempt to run a GUI.
In this case command line options are ignored.

Options:
  --version             show program's version number and exit
  -h, --help            show this help message and exit
  -i IP, --ip=IP        IP address of the server (e.g., 127.0.0.1 or ::1);
                        Note: This value is ignored when using the
                        'interfaces' option.
  -p PORT, --port=PORT  port of server (8000)
  -a PASSWORD, --password=PASSWORD
                        password to be used for administration (use -a
                        "<recycle>" to reuse the last password))
  -c SSL_CERTIFICATE, --ssl_certificate=SSL_CERTIFICATE
                        file that contains ssl certificate
  -k SSL_PRIVATE_KEY, --ssl_private_key=SSL_PRIVATE_KEY
                        file that contains ssl private key
  --ca-cert=SSL_CA_CERTIFICATE
                        Use this file containing the CA certificate to
                        validate X509 certificates from clients
  -d PID_FILENAME, --pid_filename=PID_FILENAME
                        file to store the pid of the server
  -l LOG_FILENAME, --log_filename=LOG_FILENAME
                        file to log connections
  -n NUMTHREADS, --numthreads=NUMTHREADS
                        number of threads (deprecated)
  --minthreads=MINTHREADS
                        minimum number of server threads
  --maxthreads=MAXTHREADS
                        maximum number of server threads
  -s SERVER_NAME, --server_name=SERVER_NAME
                        server name for the web server
  -q REQUEST_QUEUE_SIZE, --request_queue_size=REQUEST_QUEUE_SIZE
                        max number of queued requests when server unavailable
  -o TIMEOUT, --timeout=TIMEOUT
                        timeout for individual request (10 seconds)
  -z SHUTDOWN_TIMEOUT, --shutdown_timeout=SHUTDOWN_TIMEOUT
                        timeout on shutdown of server (5 seconds)
  --socket-timeout=SOCKET_TIMEOUT
                        timeout for socket (5 second)
  -f FOLDER, --folder=FOLDER
                        location of the applications folder (also known as directory)
  -v, --verbose         increase --test verbosity
  -Q, --quiet           disable all output
  -D DEBUGLEVEL, --debug=DEBUGLEVEL
                        set debug output level (0-100, 0 means all, 100 means
                        none; default is 30)
  -S APPNAME, --shell=APPNAME
                        run web2py in interactive shell or IPython (if
                        installed) with specified appname (if app does not
                        exist it will be created). APPNAME like a/c/f (c,f
                        optional)
  -B, --bpython         run web2py in interactive shell or bpython (if
                        installed) with specified appname (if app does not
                        exist it will be created). Use combined with --shell
  -P, --plain           only use plain python shell; should be used with
                        --shell option
  -M, --import_models   auto import model files; default is False; should be
                        used with --shell option
  -R PYTHON_FILE, --run=PYTHON_FILE
                        run PYTHON_FILE in web2py environment; should be used
                        with --shell option
  -K SCHEDULER, --scheduler=SCHEDULER
                        run scheduled tasks for the specified apps: expects a
                        list of app names as -K app1,app2,app3 or a list of
                        app:groups as -K app1:group1:group2,app2:group1 to
                        override specific group_names. (only strings, no
                        spaces allowed. Requires a scheduler defined in the
                        models
  -X, --with-scheduler  run schedulers alongside webserver
  -T TEST_PATH, --test=TEST_PATH
                        run doctests in web2py environment; TEST_PATH like
                        a/c/f (c,f optional)
  -C, --cron            trigger a cron run manually; usually invoked from a
                        system crontab
  --softcron            triggers the use of softcron
  -Y, --run-cron        start the background cron process
  -J, --cronjob         identify cron-initiated command
  -L CONFIG, --config=CONFIG
                        config file
  -F PROFILER_DIR, --profiler=PROFILER_DIR
                        profiler dir
  -t, --taskbar         use web2py gui and run in taskbar (system tray)
  --nogui               text-only, no GUI
  -A ARGS, --args=ARGS  should be followed by a list of arguments to be passed
                        to script, to be used with -S, -A must be the last
                        option
  --no-banner           Do not print header banner
  --interfaces=INTERFACES
                        listen on multiple addresses: "ip1:port1:key1:cert1:ca
                        _cert1;ip2:port2:key2:cert2:ca_cert2;..."
                        (:key:cert:ca_cert optional; no spaces; IPv6 addresses
                        must be in square [] brackets)
  --run_system_tests    runs web2py tests
```

## Flujo de trabajo
- Una petición HTTP llega al servidor web.
- La petición se traduce y se pasa al enrutador.
- El enrutador elije la aplicación que recibirá la petición y traduce el PATH_INFO de la URL a su función correspondiente. Cada URL tiene su función/acción correspondiente.
- Las peticiones de ficheros que están en el directorio static se gestionan directamente y los ficheros grandes se envían directamente al cliente por streaming.
- Cualquier otra petición corresponderá a una acción, es decir una función de un controlador de la aplicación seleccionada.
- Antes de ejecutar la acción:
  - Si la cabecera de la petición contiene una cookie de sesión para la aplicación, se recupera el objeto de sesión. Si no la tiene, se crea un nuevo objeto de sesión.
  - Se crea un entorno de ejecución y dentro se ejecutan los modelos.
  - Por último, se ejecuta la acción en ese mismo entorno.
    - Si la acción devuelve un string, este se devuelve al cliente.
    - Si la acción devuelve un iterable, se itera sobre el mismo y se genera un stream para el cliente.
    - Si la acción devuelve un diccionario, Web2py trata de renderizarlo en una vista.
    - La vista debe tener el mismo nombre que la acción y la misma extensión que la solicitud.
    - Si no existe, Web2py elegirá una vista genérica si existe y está habilitada.
    - La vista dispondrá de todos los objetos definidos en los modelos, además del diccionario pasado por la acción, pero no de las variables globales definidas en el controlador.
  - Todo el código se ejecuta en una única transacción de base de datos, a no ser que se quiera otra cosa.
    - Si no hay errores, la transacción es cerrada (commit).
    - Si los hay, el error se guarda en un ticket, que se muesta al usuario. Únicamente el administrador del sistema puede leer la traza del ticket.

### Además:
- Los modelos en el mismo directorio se ejecutan en orden alfabético.
- Cualquier variable definida en un modelo será visible por los siguientes modelos en orden de ejecución, por los controladores y por las vistas.

### Modelos condicionales
Si los modelos se organizan en subdirectorios, se ejecutarán de forma condicional, en base al controlador. Esto evita procesar todas las tablas a cada petición. Por ejemplo, si se recibe la petición a/c/f, donde **a** es la aplicación, **c** el controlador y **f** la función, el orden de ejecución de los modelos será el siguiente:
1. applications/a/models/*.py
2. applications/a/models/c/*.py
3. applications/a/models/c/f/*.py

Este es el comportamiento por defecto. Modificando la lista response.models_to_run, se puede forzar el comportamiento deseado.

Se ejecuta el controlador y se llama a la función, lo que implica que todo el código global del controlador es ejecutado con cada petición.

La vista es llamada únicamente si la acción devuelve un diccionario. Si la vista no se encuentra, Web2py trata de utilizar una genérica. Por defecto, las vistas genéricas están deshabilitadas, a no ser que la petición sea local. Se puede ver en /models/db.py de la aplicación welcome.

### Acciones
Las acciones son las funciones "sin parámetros" definidas en los controladores. Los posibles comportamientos de una acción son:
```python
# devolver un string
def index():
    return 'Hola'

# devolver un diccionario para una vista
def index():
    return dict(clave='valor')

# devolver todas las variables locales de la acción
# locals() es un diccionario que contiene todas las variables locales
def index():
    return locals()

# redirigir al cliente a otra página
def index():
    redirect(URL('otra_accion'))

# devolver un ayudante HTTP, distinto de "200 OK":
def index():
    raise HTTP(404)

# devolver un ayudante HTML
# generalmente para AJAX y componentes
def index():
    return FORM(INPUT(_name='test'))

# si devuelve un diccionario
# este puede contener ayudantes HTML
def index():
    return dict(form=SQLFORM.factory(Field('nombre')).process())
```

## Despacho de peticiones
![image](http://www.web2py.com/books/default/image/29/en5700.png)

Pero no es obligatorio que la URL esté completa. Podemos obviar tanto la acción, como el controlador y la aplicación.
En esos casos, Web2py establece unos valores por defecto:
- acción (f): index
- controlador (c): default.py
- aplicación (a): init. Si no existe lo intenta con welcome

Esto no es así para las peticiones del tipo application/static/filename. Web2py entiende que es una petición de un fichero "filename" en el directorio "static" de la aplicación "application".

En estos casos Web2py no crea una sesión, ni una cookie, ni ejecuta los modelos. Web2py siempre hace streaming de los ficheros en trozos de 1MB.

Web2py soporta el protocolo IF_MODIFIED_SINCE y no envía el fichero si ya está en la caché del navegador y no ha cambiado.

Cuando se enlaza a un fichero de audio o vídeo en un directorio static, si quieres forzar la descarga en lugar del streaming, hay que añadir ?attachment a la URL. Esto le dice a Web2py que ponga la cabecera Content-Disposition de la respuesta a "attachment"
```html
<a href="/app/static/my_audio_file.mp3?attachment">Download</a>
```

Web2py envía las peticiones GET/POST de la forma: a/c/f.html/x/y/z?p=1&q=2 a la función **f**, del controlador **c.py**, de la aplicación **a** y guarda el resto de parámetros de la URL:
```python
request.application = 'a'
request.controller = 'c'
request.function = 'f'
request.args = ['x', 'y', 'z']
request.vars = {'p': 1, 'q': 2}
```
request.url: guarda la URL completa de la petición, sin incluir las variables GET

request.ajax: por defecto es False y determina si la petición llegó vía AJAX

request.cid: componente, si la petición es AJAX

request.env.path_info: 'a/c/f'

request.env.http_host: '127.0.0.1:8000'

Únicamente están permitidas la URLs que contienen:
- Caracteres alfanuméricos
- Subrayados
- Barras /
- Los argumentos pueden contener puntos, no consecutivos
- Los espacios se sustituyen por _ antes de validarse
- Si la URL no es válida, Web2py devuelve un error HTTP 400

## [API](http://www.web2py.com/books/default/chapter/29/04/the-core#API) de Web2py

Los modelos, los controladores y las vistas son "ejecutados" en un entorno donde los siguientes objetos "**están disponibles siempre**", sin necesidad de importarlos:

**Objetos globales**:
- request, response, session, cache

**Internacionalización**
- T

**Navegación**:
- redirect, HTTP

**Ayudantes**:
- XML, URL, BEAUTIFY
- A, B, BODY, BR, CENTER, CODE, COL, COLGROUP, DIV, EM, EMBED, FIELDSET, FORM, H1, H2, H3, H4, H5, H6, HEAD, HR, HTML, I, IFRAME, IMG, INPUT, LABEL, LEGEND, LI, LINK, OL, UL, META, OBJECT, OPTION, P, PRE, SCRIPT, OPTGROUP, SELECT, SPAN, STYLE, TABLE, TAG, TD, TEXTAREA, TH, THEAD, TBODY, TFOOT, TITLE, TR, TT, URL, XHTML, xmlescape, embed64
- CAT, MARKMIN, MENU, ON

**Formularios y tablas**:
- SQLFORM (SQLFORM.factory, SQLFORM.grid, SQLFORM.smartgrid)

**Validadores**:
- CLEANUP, CRYPT, IS_ALPHANUMERIC, IS_DATE_IN_RANGE, IS_DATE, IS_DATETIME_IN_RANGE, IS_DATETIME, IS_DECIMAL_IN_RANGE, IS_EMAIL, IS_EMPTY_OR, IS_EXPR, IS_FLOAT_IN_RANGE, IS_IMAGE, IS_IN_DB, IS_IN_SET, IS_INT_IN_RANGE, IS_IPV4, IS_LENGTH, IS_LIST_OF, IS_LOWER, IS_MATCH, IS_EQUAL_TO, IS_NOT_EMPTY, IS_NOT_IN_DB, IS_NULL_OR, IS_SLUG, IS_STRONG, IS_TIME, IS_UPLOAD_FILENAME, IS_UPPER, IS_URL

**Base de datos**:
- DAL, Field

### Acceso a la API desde otros módulos Python
Los modelos y controladores pueden importar módulos Python. Estos módulos, a su vez, pueden necesitar aceso a la API de Web2py. Para ello:
```python
from gluon import *
```
Otra posibilidad es la de incluir Web2py en sys.path

### Objeto current
Web2py define varios objetos globales (request, response, session, cache y T) que existen únicamente cuando se produce una petición HTTP. Los módulos/ficheros del directorio modules, no tienen acceso a ellos. Para que lo tengan se utiliza el objeto current:
```python
# /myapp/modules/mytest.py
from gluon import *
def ip(): return current.request.client

# desde cualquier controlador de myapp
import mytest
def index():
    return "Tu dirección IP es " + mytest.ip()
```
Cuando se importa mytest, se busca primero en el directorio modules de la aplicación y después en sys.path.

## [request](http://www.web2py.com/books/default/chapter/29/04/the-core#request)
Es el objeto que encapsula todas las características de una petición en el entorno HTTP-Web2py.

## [response](http://www.web2py.com/books/default/chapter/29/04/the-core#response)
Encapsula las características de la respuesta.

## [session](http://www.web2py.com/books/default/chapter/29/04/the-core#session)
Objeto para la gestión de sesiones.

## [HTTP y redirect](http://www.web2py.com/books/default/chapter/29/04/the-core#HTTP-and-redirect)
Web2py define en su código una única excepción, HTTP, que puede lanzarse en cualquier modelo, controlador o vista.
Para ello:
```python
raise HTTP(400, 'Mensaje a mostrar en el cuerpo')
```

Web2py define un atajo para la redirección:
```python
redirect(url)
# es un atajo de raise HTTP(303, 'Has sido redirigido <a href="%s">' % url)
```

## [Internacionalización y pluralización](http://www.web2py.com/books/default/chapter/29/04/the-core#Internationalization-and-Pluralization-with-T)
El objeto T es el encargado de la traducción.
```python
mensaje = T('Hello')
# traduce la cadena 'Hello' al lenguaje solicitado por la petición
```

### Interpolación
```python
mensaje = T('Hello %(nombre), how are you?', dict(nombre='Antonio'))
```
### ¿Lenguaje?
El lenguaje al que se traduce, viene determinado por la petición, pero puede forzarse:
```python
# lenguaje de la petición
# language = request.env.http_accept_language
T.force('es-es')   # el lenguaje al que se traducirá será el español
# también se puede forzar únicamente para una cadena en concreto
T('Hello', language='es-es')
# es posible desactivar el sistema de traducción
T.force(None)
```

## Logging
Web2py utiliza el módulo estándar [logging](http://docs.python.org/library/logging.html). Por ejemplo, en un modelo:
```python
import logging
logger = logging.getLogger("web2py.app.app1")
logger.setLevel(logging.DEBUG)
```
Una vez definido, es posible utilizarlo en cualquier modelo, controlador o vista que se ejecute posteriormente.
```
logger.debug('Depurando')
logger.info('Informando')
logger.warn('Alerta')
logger.error('Error')
```
En el directorio examples está el fichero "logging.example.conf" que contiene información muy interesante. Es muy sencillo modificarlo a tu conveniencia y dejarlo en el directorio raíz de Web2py con el nombre "logging.conf".

## Nueva aplicación (app2)
Vamos a seguir conociendo el framework, creando una nueva aplicación, sin modelos (BB.DD). Únicamente utilizaremos controladores y vistas.

### Primer controlador (c1)
```python
# un controlador es un fichero Python
# contiene acciones que responden a URLs
# app/controlador/accion
# pero puede contener más cosas

# importo módulos python
import logging
logger = logging.getLogger("web2py.app.app1")
logger.setLevel(logging.DEBUG)

# logging del controlador
logger.debug(request.env.path_info)

# aquí declaro un varios objetos
# a nivel global del controlador
# estarán disponibles para todas sus acciones
STRING = 'Esto es un texto'
NUMBER = 123
LIST = [1, 2, 3]

# una acción es una "función sin parámetros que devuelve algo"
# si lo que devuelve se puede serializar como una cadena de texto
#   la respuesta será el objeto serializado
# si devuelve un diccionario se trata de renderizar mediante una vista
def a1():
    'app/res/a1 -> Hola'

    # comprueba el código fuente generado en el navegador
    return 'Hola'

def a2():
    'app/res/a2 -> STRING'

    # comprueba el código fuente generado en el navegador
    return STRING

def a3():
    'app/res/a3 -> NUMBER'

    # comprueba el código fuente generado en el navegador
    return NUMBER

# ERROR!!!!!
def a4():
    'app/res/a4 -> LIST'

    # esta acción provocará un error
    return LIST

def html1():
    'app/res/html1 -> html'

    # comprueba el código fuente generado en el navegador
    return "<html><body><h1>Hola</h1></body></html>"

def html2():
    'app/res/html -> html'

    # comprueba el código fuente generado en el navegador
    return "<html><body><h1>{0}</h1></body></html>".format('Hola, de nuevo')

def a5():
    'app/res/a5 -> diccionario'

    # como devuleve un diccionario se renderiza con una vista
    # Web2py busca la vista /app/views/controlador/accion.html
    # en esta caso: /app1/views/c1/a5.html
    # si no existe se usa la vista genérica, si así se desea
    # comprueba el código fuente generado en el navegador
    return {'a': 1, 'b': 'Hola', 'c': type}
```

### Segundo controlador (c2.py)
```python
def a1():
    # locals() es un diccionario
    # contiene las variables locales de la función

    # request es uno de los objetos globales de Web2py
    # contiene información relativa a la petición

    entorno = request.env
    variables = request.vars
    argumentos = request.args

    # haz la siguiente petición
    # http://127.0.0.1:8000/app1/c2/a1/1/2/3?a=hola&b=123
    #   path_info: /app1/c2/a1/1/2/3
    #   query_string: a=hola&b=123
    #   variables: [1, 2, 3]
    #   argumentos: {'a': 'hola', 'b': '123'}

    return locals()

def a2():
    message = 'Hola, soy Web2py'

    # Web2py buscará la vista que le corresponde a esta acción
    #   /app1/views/c2/a2.html
    # vamos a crearla desde cero

    return locals()

def a3():
    message = 'Hola, de nuevo'

    # vamos a crear una plantilla apoyándonos en el entorno
    # la plantilla a3.html extederá layout.html
    # echaremos un vistazo a layout.html
    return locals()
```

### Tercer controlador (c3.py)
```python
def a1():

    message = 'Sesiones'

    # session es uno de los objetos globales de Web2py
    # sirve para guardar las variables de sesión
    # dichas variables sobreviven al ciclo "petición -> respuesta"
    s = session

    return locals()

def a2():
    message = 'Sesiones'

    # creo una variable de sesión
    # y le asigno el valor 1
    contador = session.contador = 1

    return locals()

def a3():

    # creo una variable de sesión
    # voy a contar el número de veces que el usuario carga la página

    # if session.contador:
    #     session.contador += 1
    # else:
    #     session.contador = 1
    # session.contador = 1 if not session.contador else session.contador + 1
    # session.contador = session.get('contador', 0) + 1
    session.contador = (session.contador or 0) + 1
    message = 'Contador: {}'.format(session.contador)

    # prueba a abrir varias ventanas privadas que apunten a esta URL
    # http://127.0.0.1:8000/app1/c3/a3

    return locals()

def a4():

    # Web2py define un objeto global T como ayuda a la internacionalización
    # se encarga de traducir un texto al lenguaje que solicita el cliente
    # las traducciones están en /app/languages/lenguaje.py
    lenguaje = request.env.http_accept_language
    message = T('Welcome')

    # prueba a recargar esta página cambiando el idioma por defecto

    return locals()
```

### Cuarto controlador (c4.py)
```python
def a1():
    'redirige a google.com'

    redirect('http://google.com')

def a2():
    'ayudante para la construcción de URLs'

    # URL(a='a', c='c', f='f', args=['x', 'y'], vars=dict(z='t'))
    # la firma real de URL es más complicada
    # echa un vistazo al código fuente en gluon/html.py

    # se redirige a la acción 'a3'
    redirect(URL('a3', args=['aa', 'bb', 'cc'], vars={'x': 1, 'y': 2}))

def a3():
    return dict(args=request.args, vars=request.vars)

def a4():
    'el formulario está en la vista c4/a4.html'
    return locals()

def a5():
    'el formulario está en la vista c4/a5.html'
    return locals()

def a6():
    'ahora genero el formulario con Web2py'

    form = SQLFORM.factory(Field('name'))
    # comprueba la vista asociada para ver cómo se renderiza
    return locals()

def a7():
    'la acción a6 no sirve de mucho'

    # crea un formulario para mi
    form = SQLFORM.factory(Field('name'))
    # form = SQLFORM.factory(Field('name', requires=IS_NOT_EMPTY()))

    # procesa el formulario
    form.process()  # ¿qué?

    if form.accepted:
        # si el formulario se envió correctamente
        response.flash = 'Formulario procesado correctamente'
        # el mensaje no se muestra
        # la redirección inicia un nuevo ciclo petición -> respuesta
        # session.flash = 'Formulario procesado correctamente'
        # redirijo el flujo a otra acción y envío el nombre
        # guardado en el formulario
        redirect(URL('a3', vars={'name': form.vars.name}))
    elif form.errors:
        # si se ha producido algún error
        response.flash = 'Formulario procesado con errores'
    else:
        # por aquí pasa cuando se crea la primera vez
        response.flash = 'Formulario creado'

    return locals()
```